In [1]:
import pandas
import datetime
import geomag
import numpy

In [2]:
# some constants
cutOffLosVel = 10.

In [3]:
#### In this block we load Velocity data ####
#### In this block we load Velocity data ####
#### In this block we load Velocity data ####
# need the formatted velocity file to read data
outFormattedFile = "/home/bharat/Documents/code/new-vel-data/veldata/saps-vels-20110723.txt"#"../data/new-test-vels-north.txt"
# os.system( 'cat ' + rawVelFileName + ' | tr -s "  " > ' + outFormattedFile )
inpColNames = [ "dateStr", "timeStr", "beam", "range", \
          "geoAzm", "azimCalcMag", "vLos", "spwdth", "pwr", "MLAT", "MLON", "MLT",  \
          "GLAT", "GLON", "radId", "radCode"]
velsDataDF = pandas.read_csv(outFormattedFile, delim_whitespace=True,\
                             header=None, names=inpColNames)
# add a datetime col
velsDataDF["date"] = pandas.to_datetime( \
                        velsDataDF['dateStr'].astype(str) + "-" +\
                        velsDataDF['timeStr'].astype(str), format='%Y%m%d-%H%M')
# for some reason MLAT is a str type, convert it to float
velsDataDF["MLAT"] = velsDataDF["MLAT"].astype(float)
# Also get a normMLT for plotting
velsDataDF['normMLT'] = [x-24 if x >= 12 else x for x in velsDataDF['MLT']]
velsDataDF.head()
#### In this block we load SAPS Velocity data ####
#### In this block we load SAPS Velocity data ####
#### In this block we load SAPS Velocity data ####

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT
0,20110723,630,0,8,-59.02,-65.8018,0.0000,3.5227,12.8294,50.8287,-39.3022,22.9439,40.9339,-104.7630,204,fhw,2011-07-23 06:30:00,-1.0561
1,20110723,630,0,9,-59.02,-66.9103,0.0000,4.1267,12.8294,50.9404,-39.9338,22.9018,41.1292,-105.2415,204,fhw,2011-07-23 06:30:00,-1.0982
2,20110723,630,0,13,-59.02,-71.3813,-62.0701,7.8932,12.9605,51.0749,-41.9021,22.7706,41.5285,-106.7663,204,fhw,2011-07-23 06:30:00,-1.2294
3,20110723,630,0,14,-59.02,-72.3248,-64.0840,23.0972,14.5189,51.1041,-42.3912,22.7380,41.6252,-107.1473,204,fhw,2011-07-23 06:30:00,-1.2620
4,20110723,630,0,15,-59.02,-72.9329,-56.0960,66.0415,16.8201,51.2194,-43.0455,22.6944,41.8324,-107.6470,204,fhw,2011-07-23 06:30:00,-1.3056


In [4]:
# convert azimuth to magn coords
gm = geomag.geomag.GeoMag()
def convert_to_mag_azm(row):
#     print type(row["date"].to_datetime()),row["date"].to_datetime()
    mg = gm.GeoMag(row['GLAT'], \
                   row['GLON'], h=300., time=row['date'].date())
    azm_mag = (round(row['geoAzm'] - mg.dec,2))
    return azm_mag


velsDataDF["magAzm"] = -999.#velsDataDF.apply( convert_to_mag_azm, axis=1 )
velsDataDF.head()

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,magAzm
0,20110723,630,0,8,-59.02,-65.8018,0.0000,3.5227,12.8294,50.8287,-39.3022,22.9439,40.9339,-104.7630,204,fhw,2011-07-23 06:30:00,-1.0561,-999.0
1,20110723,630,0,9,-59.02,-66.9103,0.0000,4.1267,12.8294,50.9404,-39.9338,22.9018,41.1292,-105.2415,204,fhw,2011-07-23 06:30:00,-1.0982,-999.0
2,20110723,630,0,13,-59.02,-71.3813,-62.0701,7.8932,12.9605,51.0749,-41.9021,22.7706,41.5285,-106.7663,204,fhw,2011-07-23 06:30:00,-1.2294,-999.0
3,20110723,630,0,14,-59.02,-72.3248,-64.0840,23.0972,14.5189,51.1041,-42.3912,22.7380,41.6252,-107.1473,204,fhw,2011-07-23 06:30:00,-1.2620,-999.0
4,20110723,630,0,15,-59.02,-72.9329,-56.0960,66.0415,16.8201,51.2194,-43.0455,22.6944,41.8324,-107.6470,204,fhw,2011-07-23 06:30:00,-1.3056,-999.0


In [5]:
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
# remove velocies whose magnitude is less than 200 m/s
velsDataDF = velsDataDF[ abs(velsDataDF["vLos"]) >= cutOffLosVel ]
# SAPS(westward) vLos are positive for positive azimuths and vice versa.
# filter the others out
velsDataDF = velsDataDF[ velsDataDF["azimCalcMag"]/velsDataDF["vLos"] > 0. ].reset_index(drop=True)
velsDataDF.head()
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####
#### In this block we filter for SAPS velocities ####

,dateStr,timeStr,beam,range,geoAzm,azimCalcMag,vLos,spwdth,pwr,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,magAzm
0,20110723,630,0,13,-59.02,-71.3813,-62.0701,7.8932,12.9605,51.0749,-41.9021,22.7706,41.5285,-106.7663,204,fhw,2011-07-23 06:30:00,-1.2294,-999.0
1,20110723,630,0,14,-59.02,-72.3248,-64.0840,23.0972,14.5189,51.1041,-42.3912,22.7380,41.6252,-107.1473,204,fhw,2011-07-23 06:30:00,-1.2620,-999.0
2,20110723,630,0,15,-59.02,-72.9329,-56.0960,66.0415,16.8201,51.2194,-43.0455,22.6944,41.8324,-107.6470,204,fhw,2011-07-23 06:30:00,-1.3056,-999.0
3,20110723,630,0,16,-59.02,-73.5450,-36.1537,86.8168,16.0908,51.3281,-43.6987,22.6508,42.0338,-108.1474,204,fhw,2011-07-23 06:30:00,-1.3492,-999.0
4,20110723,630,0,17,-59.02,-74.1585,-36.1537,100.3948,14.3879,51.4306,-44.3511,22.6073,42.2299,-108.6488,204,fhw,2011-07-23 06:30:00,-1.3927,-999.0


In [6]:
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
datFileName = "../data/processedSaps.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ', dtype={'dateStr':'str', 'time': 'str'})

sapsDataDF["date"] = pandas.to_datetime( \
                        sapsDataDF['dateStr'] + "-" +\
                        sapsDataDF['time'], format='%Y%m%d-%H%M')
sapsDataDF.head()
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####
#### Need prev SAPS data to determine SAPS scatter ####

,dateStr,sapsLat,sapsMLT,sapsVel,radId,poesLat,poesMLT,dst_date,dst_index,time,date
0,20110107,56.5,17.7543,308.2077,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
1,20110107,55.5,18.0147,224.1588,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
2,20110107,56.5,17.8749,307.4328,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
3,20110107,55.5,18.1324,222.4787,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07
4,20110107,56.5,17.9955,305.4201,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0000,2011-01-07


In [7]:
velsDataDF["date"].unique()

array(['2011-07-23T06:30:00.000000000', '2011-07-23T06:32:00.000000000',
       '2011-07-23T06:34:00.000000000', '2011-07-23T06:36:00.000000000',
       '2011-07-23T06:38:00.000000000', '2011-07-23T06:40:00.000000000',
       '2011-07-23T06:42:00.000000000', '2011-07-23T06:44:00.000000000',
       '2011-07-23T06:46:00.000000000', '2011-07-23T06:48:00.000000000',
       '2011-07-23T06:50:00.000000000', '2011-07-23T06:52:00.000000000',
       '2011-07-23T06:54:00.000000000', '2011-07-23T06:56:00.000000000',
       '2011-07-23T06:58:00.000000000', '2011-07-23T07:00:00.000000000',
       '2011-07-23T07:02:00.000000000', '2011-07-23T07:04:00.000000000',
       '2011-07-23T07:06:00.000000000', '2011-07-23T07:08:00.000000000',
       '2011-07-23T07:10:00.000000000', '2011-07-23T07:12:00.000000000',
       '2011-07-23T07:14:00.000000000', '2011-07-23T07:16:00.000000000',
       '2011-07-23T07:18:00.000000000', '2011-07-23T07:20:00.000000000',
       '2011-07-23T07:22:00.000000000', '2011-07-23

In [8]:

#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
# we choose 0840 UT since that time was analyzed in the paper
timeSlctd = datetime.datetime( 2011, 7, 23, 7, 26 )
velAnlysDF = velsDataDF[ velsDataDF["date"] == timeSlctd ].reset_index(drop=True)
sapsSelPrdDF = sapsDataDF[  ( sapsDataDF["date"] - timeSlctd < numpy.timedelta64(40,'m') )\
           & ( sapsDataDF["date"] - timeSlctd > numpy.timedelta64(0,'m') )  ].reset_index(drop=True)
# Now check if there is more than one time period in the selected interval
# ideally there shouldn't be, but if we find one we should do something
# about it (like take an average)...for now its undecided.!
if len( sapsSelPrdDF["time"].unique() ) > 1:
    print "There is more than one time period selected!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
    print "NEED TO CHECK SOMETHING WRONG!!!"
poesBndDF = sapsSelPrdDF[ ["poesMLT", "poesLat"] \
                    ].drop_duplicates().reset_index(drop=True)
# Have a normalized MLT for ease of comparison
poesBndDF['normMLT'] = [x-24 if x >= 12 else x for x in poesBndDF['poesMLT']]
poesBndDF.head()
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####
#### Choose a time to and get POES boundaries as well ####

There is more than one time period selected!!!
NEED TO CHECK SOMETHING WRONG!!!
NEED TO CHECK SOMETHING WRONG!!!
NEED TO CHECK SOMETHING WRONG!!!


,poesMLT,poesLat,normMLT
0,0.0,60.2648,0.0
1,1.0,60.2495,1.0
2,21.0,63.2161,-3.0
3,22.0,61.8004,-2.0
4,23.0,60.7938,-1.0


In [9]:
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
# Merge POES boundary DF with the vels DF
velAnlysDF["normMLTRound"] = velAnlysDF["normMLT"].astype(int)
print "shape before join--->", velAnlysDF.shape
velAnlysDF = pandas.merge( velAnlysDF, poesBndDF, left_on="normMLTRound", right_on="normMLT", how="inner" )
print "shape after join--->", velAnlysDF.shape
# Filter out velocties above the POES boundary
velAnlysDF = velAnlysDF[ velAnlysDF["MLAT"] < velAnlysDF["poesLat"]\
                       ].reset_index(drop=True).drop_duplicates()
print "shape after filtering by boundary--->", velAnlysDF.shape
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####
#### For the chosen time get velocities which fall below POES BND ####

shape before join---> (1364, 20)
shape after join---> (2610, 23)
shape after filtering by boundary---> (2198, 23)


In [10]:
print velAnlysDF

       dateStr  timeStr  beam  range  geoAzm  azimCalcMag      vLos    spwdth  \
0     20110723      726     0      0  -59.02     -48.9651  -65.2103    9.2449   
1     20110723      726     0      0  -59.02     -48.9651  -65.2103    9.2449   
2     20110723      726     0      1  -59.02     -52.7775  -64.7805   11.6335   
3     20110723      726     0      1  -59.02     -52.7775  -64.7805   11.6335   
4     20110723      726     0      2  -59.02     -55.7327  -66.4007   12.1331   
5     20110723      726     0      2  -59.02     -55.7327  -66.4007   12.1331   
6     20110723      726     0      3  -59.02     -58.1100  -71.8733   12.1331   
7     20110723      726     0      3  -59.02     -58.1100  -71.8733   12.1331   
8     20110723      726     0      4  -59.02     -60.0815  -73.3707   10.1462   
9     20110723      726     0      4  -59.02     -60.0815  -73.3707   10.1462   
10    20110723      726     0     15  -59.02     -72.9329  -46.3022   35.6005   
11    20110723      726     

In [11]:
# save the data in a csv for lshell fitting
outSelCols = [ "beam", "range", "geoAzm", "azimCalcMag", "magAzm", "vLos"\
           , "MLAT", "MLT", "MLON", "GLAT", "GLON", "radId"\
           , "radCode","normMLT_x", "normMLTRound", "spwdth", "pwr" ]
velAnlysDF.to_csv( "../data/test-losVels.txt", columns=outSelCols, index=False )